# Interactive Brokers Connection Example

This notebook demonstrates basic connection and operations with Interactive Brokers using the ib_async library.

## Connection

**Important Note**: From now on, use `ib_async` instead of the older Interactive Brokers API.

In [44]:
from ib_async import *
util.startLoop()  # required in Jupyter (interactive environments), not in scripts

In [45]:
ib = IB()

**Make sure you are logged in your TWS!!**

In [46]:
ib.connect()

Error 326, reqId -1: Unable to connect as the client id is already in use. Retry with a unique client id.
Peer closed connection. clientId 1 already in use?
API connection failed: TimeoutError()


TimeoutError: 

In [ ]:
ib.positions()

[]

In [ ]:
ib.disconnect()

'Disconnecting from 127.0.0.1:7497, 117 B sent in 8 messages, 18.9 kB received in 398 messages, session time 10.5 s.'

## Contracts (Introduction)

Working with contracts is fundamental in Interactive Brokers. A contract represents a financial instrument and contains all the information needed to uniquely identify it on the exchange.

Here we'll create and qualify a Forex contract. The `qualifyContracts()` function fills in missing contract details by querying Interactive Brokers' database.

In [ ]:
from ib_async import *
util.startLoop()

In [ ]:
ib = IB()

In [ ]:
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [ ]:
contract = Forex("EURUSD")
contract

Forex('EURUSD', exchange='IDEALPRO')

Create a Forex contract for EUR/USD. The `Forex()` constructor creates a contract object, but it's not yet validated with IB:

In [ ]:
contract = ib.qualifyContracts(contract)
contract

[Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')]

The `qualifyContracts()` function validates the contract with IB and fills in missing details like contract ID, exchange info, etc.:

In [ ]:
type(contract)

list

Check the type - `qualifyContracts()` returns a list, even for a single contract:

In [ ]:
contract[0]

Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')

Access the first (and only) contract in the returned list to see the full contract details:

In [ ]:
contract[0].conId

12087792

The `conId` (contract ID) is a unique identifier assigned by Interactive Brokers to each financial instrument:

## Contract Types

Interactive Brokers supports various contract types. Each type has specific parameters:

- **Contract(conId)**: Direct reference using IB's contract ID
- **Stock**: Equity securities (symbol, exchange, currency, primaryExchange optional)
- **Forex**: Currency pairs (currency pair like 'EURUSD')
- **CFD**: Contracts for Difference (symbol)
- **Future**: Futures contracts (symbol, expiry date, exchange)
- **Option**: Options contracts (symbol, expiry, strike, right, exchange)
- **Bond**: Fixed income securities (using ISIN or other identifiers)

In [ ]:
# Contract(conId=270639)
# Stock('AMD', 'SMART', 'USD')
# Stock('INTC', 'SMART', 'USD', primaryExchange='NASDAQ')
# Forex('EURUSD')
# CFD('IBUS30')
# Future('ES', '20180921', 'GLOBEX')
# Option('SPY', '20170721', 240, 'C', 'SMART')
# Bond(secIdType='ISIN', secId='US03076KAA60')

### Handling Ambiguous vs Unambiguous Contracts

When creating contracts, some may be **ambiguous** (multiple matches) while others are **unambiguous** (single match):

- **Unambiguous**: When you provide enough details (symbol + exchange + currency), IB can uniquely identify the contract
- **Ambiguous**: When details are incomplete (e.g., just symbol), IB may find multiple matching contracts

The `qualifyContracts()` function behaves differently in each case:

In [ ]:
contract = Stock("AAPL", "SMART", "USD")
contract

Stock(symbol='AAPL', exchange='SMART', currency='USD')

**Unambiguous case**: Full contract details provided (symbol, exchange, currency):

In [ ]:
contract = ib.qualifyContracts(contract)  # unambiguous
contract

This should return a single qualified contract since the details uniquely identify AAPL:

In [ ]:
contract = ib.qualifyContracts(Stock("AAPL"))  # ambiguous
contract

**Ambiguous case**: Only symbol provided, no exchange or currency specified. This may return multiple contracts if AAPL trades on different exchanges or in different currencies:

In [ ]:
ib.disconnect()

Finally, disconnect from Interactive Brokers when done:

# Current Market Data

This section demonstrates how to retrieve real-time market data for financial instruments using Interactive Brokers.

In [ ]:
import pandas as pd
from ib_async import *
util.startLoop()

In [ ]:
ib = IB()

In [ ]:
ib.connect()

Error 326, reqId -1: Unable to connect as the client id is already in use. Retry with a unique client id.
Peer closed connection. clientId 1 already in use?
API connection failed: TimeoutError()


TimeoutError: 

In [ ]:
contract = Forex("EURUSD")
contract = ib.qualifyContracts(contract)[0]  # Qualify and get first result
contract

Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')

In [ ]:
data1 = ib.reqMktData(contract)
data1

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), defaults=IBDefaults(emptyPrice=-1, emptySize=0, unset=nan, timezone=datetime.timezone.utc), created=True)

In [ ]:
data1

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2025, 11, 6, 19, 47, 20, 138836, tzinfo=datetime.timezone.utc), timestamp=1762458440.138842, minTick=1e-05, bid=1.15437, bidSize=1000000.0, ask=1.15438, askSize=2500000.0, last=1.15435, lastSize=0.0, lastTimestamp=datetime.datetime(2025, 11, 6, 19, 47, 5, tzinfo=datetime.timezone.utc), prevBidSize=2000000.0, prevAskSize=2000000.0, volume=0.0, high=1.15525, low=1.1491, close=1.1493, ticks=[TickData(time=datetime.datetime(2025, 11, 6, 19, 47, 20, 138836, tzinfo=datetime.timezone.utc), tickType=0, price=1.15437, size=1000000.0)], defaults=IBDefaults(emptyPrice=-1, emptySize=0, unset=nan, timezone=datetime.timezone.utc), created=True)

In [ ]:
data1.ask

1.15444

In [ ]:
data1.askSize

5000000.0

In [ ]:
data1.marketPrice()

1.1544750000000001

In [ ]:
data1.time

In [ ]:
data1.time

datetime.datetime(2025, 11, 6, 19, 50, 53, 16969, tzinfo=datetime.timezone.utc)

In [ ]:
pd.to_datetime(data1.time)

Timestamp('2025-11-06 19:51:39.192668+0000', tz='UTC')

In [ ]:
contract = Stock("AAPL", "SMART", "USD")
contract

Stock(symbol='AAPL', exchange='SMART', currency='USD')

In [ ]:
contract = Forex("EURUSD")
contract = ib.qualifyContracts(contract)[0]  # Qualify and get first result
contract
data2 = ib.reqMktData(contract)
data2

ConnectionError: Not connected